# Capstone Project Notebook

### This notebook consists of codes and results that are mainly used for Data Science Project in Coursera. This notebook will be shared on my github repository for this project i.e. Coursera_Capstone.

In [52]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim
import geocoder
import pgeocode

import requests
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
print('Hello Captsone Project Course')

Hello Captsone Project Course


In [7]:
url='https://en.wikipedia.org/wiki/List_of_Austin_neighborhoods'
source = requests.get(url).text

result = BeautifulSoup(source, 'lxml')#Beautiful Soup to Parse the url page

table=result.find('table') #Finds the required table area

In [33]:
#List initialization to collect the Postalcodes, Boroughs and Neighborhoods
neighborhood=[]

for tr in table.find_all('tr')[1:]: #Iterating through rows 
    th=tr.find('th')
    neighborhood.append(th.find(text=True).rstrip('\n'))

In [36]:
df_central_austin = pd.DataFrame({'Neighborhood': neighborhood})

In [38]:
df_central_austin.shape

(22, 1)

In [58]:
latitude = []
longitude = []
for neighborhood in df_central_austin['Neighborhood'].tolist():
    address = neighborhood + ', Austin'
    location = geocoder.arcgis(address)
    lat_lng = location.latlng
    latitude.append(lat_lng[0])
    longitude.append(lat_lng[1])

In [64]:
#Add Latitude and Longitude columns to the toronto dataframe
df_central_austin['Latitude']=latitude #Adding a column in the main dataframe for Latitude  

df_central_austin['Longitude']=longitude #Adding a column in the main dataframe for Longitude
df_central_austin.dropna(inplace=True)
df_central_austin.to_csv('central_austin.csv', index=False)
df_central_austin.head()

,Neighborhood,Latitude,Longitude
0,Bryker Woods,30.305032,-97.754192
1,Caswell Heights,30.307901,-97.719394
2,Downtown Austin,30.265270,-97.746470
3,Eastwoods,39.489455,-117.065602
4,Hancock,30.297150,-97.726620


In [65]:
print('The dataframe has {} neighborhoods.'.format(df_central_austin.shape[0]))

The dataframe has 22 neighborhoods.


In [114]:
#Get coordinates for Toronto
address = 'Central Austin, Texas'

geolocator = Nominatim(user_agent="austin_explorer")
location = geolocator.geocode(address)
central_austin_latitude = location.latitude
central_austin_longitude = location.longitude
print('The geograpical coordinates of Central Austin are {}, {}.'.format(austin_latitude, austin_longitude))

The geograpical coordinates of Central Austin are 30.2988785, -97.7390157.


In [115]:
# create map of Toronto using latitude and longitude values
map_austin = folium.Map(location=[central_austin_latitude, central_austin_longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_central_austin['Latitude'], df_central_austin['Longitude'], df_central_austin['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_austin)  
    
map_austin

In [73]:
CLIENT_ID = '1MPXNMPUOYDOFVVMY0RCM4GCF4QOU54JZTHX41WA0DOGQCL1' # your Foursquare ID
CLIENT_SECRET = '2BVD34TECCSGP4RXP4GHKNSCJTNTXTTTBSLPTINBZQV5MD45' # your Foursquare Secret
VERSION = '20190216' # Foursquare API version

print('Aashish Foursquare Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Aashish Foursquare Credentials:
CLIENT_ID: 1MPXNMPUOYDOFVVMY0RCM4GCF4QOU54JZTHX41WA0DOGQCL1
CLIENT_SECRET:2BVD34TECCSGP4RXP4GHKNSCJTNTXTTTBSLPTINBZQV5MD45


In [97]:
#get dataframe for one of the neighborhood. for e.g. Oakmont Heights
df_central_austin.loc[1,'Neighborhood']

'Caswell Heights'

In [89]:
neighborhood_latitude = df_central_austin.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_central_austin.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_central_austin.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Caswell Heights are 30.30790069351307, -97.7193937061208.


In [90]:
#Use foursquare api to explore the venues in Steeles West neighborhood
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
        .format(CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=1MPXNMPUOYDOFVVMY0RCM4GCF4QOU54JZTHX41WA0DOGQCL1&client_secret=2BVD34TECCSGP4RXP4GHKNSCJTNTXTTTBSLPTINBZQV5MD45&v=20190216&ll=30.30790069351307,-97.7193937061208&radius=500&limit=100'

In [91]:
results = requests.get(url).json()

In [92]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [93]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sweet Ritual,Ice Cream Shop,30.306494,-97.715292
1,Tyson's Tacos,Taco Place,30.309664,-97.715252
2,The Omelettry,Breakfast Spot,30.306571,-97.715552
3,ColdTowne Theater,Comedy Club,30.308189,-97.715465
4,Tomlinson's Feed & Pets,Pet Store,30.309915,-97.714979


In [94]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


In [98]:
 #Function to create a dataframe consisting of Neighborhood details and venue details
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
central_austin_venues = getNearbyVenues(names=df_central_austin['Neighborhood'],
                                   latitudes=df_central_austin['Latitude'],
                                   longitudes=df_central_austin['Longitude']
                                  )

Bryker Woods
Caswell Heights
Downtown Austin
Eastwoods
Hancock
Heritage
Hyde Park
Judges' Hill
Lower Waller Creek
North University
Oakmont Heights
Old Enfield
Old Pecan Street
Old West Austin
Original Austin
Original West University
Pemberton Heights
Ridgelea
Ridgetop
Rosedale
Shoal Crest
West Downtown


In [100]:
central_austin_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bryker Woods,30.305032,-97.754192,Olive & June,30.307450,-97.751046,Italian Restaurant
1,Bryker Woods,30.305032,-97.754192,Tiny Boxwoods,30.306058,-97.749789,American Restaurant
2,Bryker Woods,30.305032,-97.754192,Austin Flower Company,30.307787,-97.751224,Flower Shop
3,Bryker Woods,30.305032,-97.754192,Kerbey Lane Café,30.308030,-97.750470,Café
4,Bryker Woods,30.305032,-97.754192,Define Body & Mind: Austin,30.308680,-97.753964,Recreation Center


In [102]:
#Grouping venues by Neighborhood
central_austin_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bryker Woods,20,20,20,20,20,20
Caswell Heights,27,27,27,27,27,27
Downtown Austin,100,100,100,100,100,100
Eastwoods,3,3,3,3,3,3
Hancock,3,3,3,3,3,3
Heritage,4,4,4,4,4,4
Hyde Park,15,15,15,15,15,15
Lower Waller Creek,77,77,77,77,77,77
North University,3,3,3,3,3,3


In [103]:
print('There are {} uniques categories.'.format(len(central_austin_venues['Venue Category'].unique())))

There are 156 uniques categories.


In [105]:
# one hot encoding
central_austin_onehot = pd.get_dummies(central_austin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_austin_onehot['Neighborhood'] = central_austin_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_austin_onehot.columns[-1]] + list(central_austin_onehot.columns[:-1])
central_austin_onehot= central_austin_onehot[fixed_columns]

central_austin_onehot.head()

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,City,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Service,Fabric Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gift Shop,Gluten-free Restaurant,Golf Course,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Knitting Store,Lake,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Plaza,Smoothie Shop,Snack Place,Soup Place,South Indian Restaurant,Spa,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bryker Woods,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bryker Woods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
central_austin_onehot.shape

(497, 157)

In [107]:
central_austin_grouped = central_austin_onehot.groupby('Neighborhood').mean().reset_index()
central_austin_grouped.head()

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,City,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Service,Fabric Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gift Shop,Gluten-free Restaurant,Golf Course,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Knitting Store,Lake,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Plaza,Smoothie Shop,Snack Place,Soup Place,South Indian Restaurant,Spa,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bryker Woods,0.05,0.0,0.0,0.000000,0.0,0.05,0.0,0.000000,0.00,0.0,0.05,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.05,0.00,0.00,0.0,0.0,0.000000,0.050000,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.050000,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.05,0.00,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.05,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.00,0.10,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.05,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.100000,0.00,0.000000,0.05,0.050000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.05,0.05,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.05,0.0,0.00,0.0,0.00,0.000000,0.00,0.0,0.000000,0.00,0.0,0.000000,0.00,0.0,0.05,0.0,0.00,0.0,0.0,0.05
1,Caswell Heights,0.00,0.0,0.0,0.037037,0.0,0.00,0.0,0.037037,0.00,0.0,0.00,0.00,0.037037,0.037037,0.0,0.0,0.00,0.0,0.00,0.037037,0.0,0.00,0.00,0.00,0.0,0.0,0.074074,0.000000,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.037037,0.0,0.00,0.0,0.037037,0.037037,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.037037,0.00,0.0,0.00,0.00,0.00,0.037037,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.0,0.037037,0.037037,0.0,0.0,0.0,0.0,0.00,0.00,0.037037,0.0,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.037037,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.037037,0.0,0.0,0.000000,0.00,0.037037,0.00,0.037037,0.000000,0.0,0.037037,0.037037,0.000000,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.037037,0.0,0.0,0.037037,0.0,0.00,0.0,0.00,0.0,0.00,0.037037,0.00,0.0,0.037037,0.00,0.0,0.037037,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00
2,Downtown Austin,0.01,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.02,0.0,0.00,0.04,0.00

In [157]:
central_austin_restaurant = central_austin_grouped.copy()
central_austin_restaurant = central_austin_restaurant[central_austin_restaurant['Indian Restaurant']==0]
central_austin_restaurant.head()

,Neighborhood,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bridge,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cheese Shop,Chinese Restaurant,City,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Event Service,Fabric Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gift Shop,Gluten-free Restaurant,Golf Course,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Knitting Store,Lake,Latin American Restaurant,Laundromat,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mongolian Restaurant,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Office,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Plaza,Smoothie Shop,Snack Place,Soup Place,South Indian Restaurant,Spa,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bryker Woods,0.05,0.0,0.0,0.000000,0.0,0.05,0.0,0.000000,0.00,0.0,0.05,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.05,0.00,0.00,0.0,0.0,0.000000,0.050000,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.000000,0.0,0.00,0.0,0.000000,0.050000,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.05,0.00,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.05,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.00,0.10,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.05,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.100000,0.00,0.000000,0.05,0.050000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.05,0.05,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.05,0.0,0.00,0.0,0.00,0.000000,0.00,0.0,0.000000,0.00,0.0,0.000000,0.00,0.0,0.05,0.0,0.00,0.0,0.0,0.05
1,Caswell Heights,0.00,0.0,0.0,0.037037,0.0,0.00,0.0,0.037037,0.00,0.0,0.00,0.00,0.037037,0.037037,0.0,0.0,0.00,0.0,0.00,0.037037,0.0,0.00,0.00,0.00,0.0,0.0,0.074074,0.000000,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.037037,0.0,0.00,0.0,0.037037,0.037037,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.037037,0.00,0.0,0.00,0.00,0.00,0.037037,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.0,0.037037,0.037037,0.0,0.0,0.0,0.0,0.00,0.00,0.037037,0.0,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.037037,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.037037,0.0,0.0,0.000000,0.00,0.037037,0.00,0.037037,0.000000,0.0,0.037037,0.037037,0.000000,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.037037,0.0,0.0,0.037037,0.0,0.00,0.0,0.00,0.0,0.00,0.037037,0.00,0.0,0.037037,0.00,0.0,0.037037,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00
2,Downtown Austin,0.01,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.02,0.0,0.00,0.04,0.00

In [158]:
#Function to sort venues in descending order
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [159]:
num_top_venues = 10
indicators = ['st','nd','rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_austin_restaurant['Neighborhood']

for ind in np.arange(central_austin_restaurant.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_austin_restaurant.iloc[ind, :], 
                                                                          num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bryker Woods,Italian Restaurant,Park,Yoga Studio,Spa,Asian Restaurant,Bank,Bridal Shop,Café,Cosmetics Shop,Flower Shop
1,Caswell Heights,Bus Stop,Gym,Pool Hall,Soup Place,Shopping Plaza,Breakfast Spot,Fabric Shop,Donut Shop,Pool,Bed & Breakfast
2,Downtown Austin,Hotel,Coffee Shop,Cocktail Bar,Restaurant,New American Restaurant,Bar,Food Truck,Mexican Restaurant,Italian Restaurant,Concert Hall
3,Eastwoods,Playground,Post Office,Café,Dessert Shop,Event Service,Electronics Store,Donut Shop,Dog Run,Dive Bar,Discount Store
4,Hancock,Park,Intersection,Golf Course,Yoga Studio,Discount Store,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run


In [160]:
# set number of clusters
kclusters = 5

central_austin_restaurant_clustering = central_austin_restaurant.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_austin_restaurant_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 2, 0, 3, 3, 3, 0, 0])

In [161]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

central_austin_merged = df_central_austin

# merge scarborough_grouped with df_scarborough to add latitude/longitude for each neighborhood
central_austin_merged = central_austin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

central_austin_merged.dropna(inplace=True)
central_austin_merged.reset_index(drop=True)

central_austin_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bryker Woods,30.305032,-97.754192,3.0,Italian Restaurant,Park,Yoga Studio,Spa,Asian Restaurant,Bank,Bridal Shop,Café,Cosmetics Shop,Flower Shop
1,Caswell Heights,30.307901,-97.719394,3.0,Bus Stop,Gym,Pool Hall,Soup Place,Shopping Plaza,Breakfast Spot,Fabric Shop,Donut Shop,Pool,Bed & Breakfast
2,Downtown Austin,30.265270,-97.746470,3.0,Hotel,Coffee Shop,Cocktail Bar,Restaurant,New American Restaurant,Bar,Food Truck,Mexican Restaurant,Italian Restaurant,Concert Hall
3,Eastwoods,39.489455,-117.065602,2.0,Playground,Post Office,Café,Dessert Shop,Event Service,Electronics Store,Donut Shop,Dog Run,Dive Bar,Discount Store
4,Hancock,30.297150,-97.726620,0.0,Park,Intersection,Golf Course,Yoga Studio,Discount Store,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run


In [162]:
# create map
map_clusters = folium.Map(location=[central_austin_latitude, central_austin_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_austin_merged['Latitude'], central_austin_merged['Longitude'], 
                                  central_austin_merged['Neighborhood'], central_austin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [163]:
central_austin_merged.loc[central_austin_merged['Cluster Labels'] == 0, central_austin_merged.columns[[0] + list(range(3, central_austin_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hancock,0.0,Park,Intersection,Golf Course,Yoga Studio,Discount Store,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run
9,North University,0.0,Food & Drink Shop,Arcade,Park,Discount Store,Farmers Market,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run
10,Oakmont Heights,0.0,Park,Ice Cream Shop,Music Store,Knitting Store,Dog Run,Yoga Studio,Dive Bar,Fabric Shop,Event Service,Electronics Store


In [164]:
central_austin_merged.loc[central_austin_merged['Cluster Labels'] == 1, central_austin_merged.columns[[0] + list(range(3, central_austin_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Original Austin,1.0,Park,Yoga Studio,Dessert Shop,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run,Dive Bar,Discount Store


In [165]:
central_austin_merged.loc[central_austin_merged['Cluster Labels'] == 2, central_austin_merged.columns[[0] + list(range(3, central_austin_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Eastwoods,2.0,Playground,Post Office,Café,Dessert Shop,Event Service,Electronics Store,Donut Shop,Dog Run,Dive Bar,Discount Store


In [166]:
central_austin_merged.loc[central_austin_merged['Cluster Labels'] == 3, central_austin_merged.columns[[0] + list(range(3, central_austin_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bryker Woods,3.0,Italian Restaurant,Park,Yoga Studio,Spa,Asian Restaurant,Bank,Bridal Shop,Café,Cosmetics Shop,Flower Shop
1,Caswell Heights,3.0,Bus Stop,Gym,Pool Hall,Soup Place,Shopping Plaza,Breakfast Spot,Fabric Shop,Donut Shop,Pool,Bed & Breakfast
2,Downtown Austin,3.0,Hotel,Coffee Shop,Cocktail Bar,Restaurant,New American Restaurant,Bar,Food Truck,Mexican Restaurant,Italian Restaurant,Concert Hall
5,Heritage,3.0,Construction & Landscaping,Trail,Dog Run,Bus Stop,Discount Store,Farmers Market,Fabric Shop,Event Service,Electronics Store,Donut Shop
6,Hyde Park,3.0,Pool,Italian Restaurant,Bakery,Bus Stop,Mexican Restaurant,Park,Cheese Shop,Juice Bar,Beer Store,Sandwich Place
8,Lower Waller Creek,3.0,Food Truck,Bar,Mexican Restaurant,Coffee Shop,Dive Bar,Grocery Store,Restaurant,Lounge,Ice Cream Shop,Gym / Fitness Center
11,Old Enfield,3.0,Discount Store,Sandwich Place,City,Chinese Restaurant,Yoga Studio,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run
13,Old West Austin,3.0,Spa,Gym / Fitness Center,Yoga Studio,Gym,Food Truck,Salon / Barbershop,Café,Health & Beauty Service,Park,Office
15,Original West University,3.0,Coffee Shop,Food Truck,Sandwich Place,Sushi Restaurant,Gym,Deli / Bodega,Museum,Dessert Shop,Mexican Restaurant,Electronics Store
16,Pemberton Heights,3.0,Tennis Court,Spa,Gym,Dog Run,Outdoor Sculpture,Food Truck,Bus Station,Yoga Studio,Electronics Store,Donut Shop


In [167]:
central_austin_merged.loc[central_austin_merged['Cluster Labels'] == 4, central_austin_merged.columns[[0] + list(range(3, central_austin_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Ridgetop,4.0,Park,Lake,Music Venue,Yoga Studio,Dive Bar,Fabric Shop,Event Service,Electronics Store,Donut Shop,Dog Run
